In [1]:
import os
with open("/home/billxbf/Documents/myks/openai.key", "r") as f:
    keys = f.readlines()
    os.environ["OPENAI_API_KEY"] = keys[0].strip()
with open("/home/billxbf/Documents/myks/serpapi.key", "r") as f:
    keys = f.readlines()
    os.environ["SERPAPI_API_KEY"] = keys[0].strip()

In [2]:
from algos.PWS import *
from algos.react import ReactBase, ReactZeroShot
from algos.notool import CoT, IO
from prompts import fewshots

## REACT BASE

In [3]:
reactbase = ReactBase(fewshot=fewshots.HOTPOTQA_REACT,model_name="gpt-3.5-turbo",verbose=True)
reactbase.agent.max_iterations

8

In [4]:
response = reactbase.run("What is the name of the cognac house that makes the main ingredient in The Hennchata?")



> Entering new AgentExecutor chain...
Thought: I need to search for The Hennchata and find out which cognac house makes the main ingredient.
Action: Search[The Hennchata]
Observation: The Hennchata is a cocktail consisting of Hennessy cognac and Mexican rice horchata agua fresca. It was invented in 2013 by Jorge Sánchez at his Chaco's Mexican restaurant in San Jose, California.
Thought:The Hennchata contains Hennessy cognac, so I need to search for the cognac house Hennessy and confirm if it is the one that makes the main ingredient.
Action: Search[Hennessy cognac]
Observation: Jas Hennessy & Co., commonly known simply as Hennessy (French pronunciation: ​[ɛnɛsi]), is a French producer of cognac, which has its headquarters in Cognac, France.
It is one of the "big four" cognac houses, along with Martell, Courvoisier, and Rémy Martin, who together make around 85% of the world's cognac. Hennessy sells approximately 102 million bottles of its cognacs per year, making it the world's larges

In [5]:
response

{'wall_time': 8.863192796707153,
 'input': 'What is the name of the cognac house that makes the main ingredient in The Hennchata?',
 'output': 'Hennessy',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input='The Hennchata', log='Thought: I need to search for The Hennchata and find out which cognac house makes the main ingredient.\nAction: Search[The Hennchata]'),
   "The Hennchata is a cocktail consisting of Hennessy cognac and Mexican rice horchata agua fresca. It was invented in 2013 by Jorge Sánchez at his Chaco's Mexican restaurant in San Jose, California."),
  (AgentAction(tool='Search', tool_input='Hennessy cognac', log='The Hennchata contains Hennessy cognac, so I need to search for the cognac house Hennessy and confirm if it is the one that makes the main ingredient.\nAction: Search[Hennessy cognac]'),
   'Jas Hennessy & Co., commonly known simply as Hennessy (French pronunciation: \u200b[ɛnɛsi]), is a French producer of cognac, which has its headquarters in Cognac, 

## REACT ZERO SHOT

In [1]:
reactzero = ReactZeroShot(verbose=True)
response2 = reactzero.run("Who is the director of the 2003 film which has scenes in it filmed at the Quality Cafe in Los Angeles?")

NameError: name 'ReactZeroShot' is not defined

In [4]:
response2

{'wall_time': 8.320967435836792,
 'input': 'Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?',
 'output': 'Richard Nixon',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input='Matt Groening Milhouse', log=' I need to find out who Matt Groening named Milhouse after.\nAction: Search\nAction Input: "Matt Groening Milhouse"'),
   'Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.'),
  (AgentAction(tool='Search', tool_input='Matt Groening Milhouse origin', log=' I need to find out who Matt Groening named Milhouse after.\nAction: Search\nAction Input: "Matt Groening Milhouse origin"'),
   'Creation. Milhouse was designed by Matt Groening for a planned series on NBC, which was abandoned. The design was then used for a Butterfinger commercial, and it was decided to use the character in the series.

## PWS

In [7]:
pws = PWS_Base()
response3 = pws.run("What is the name of the cognac house that makes the main ingredient in The Hennchata?")

In [8]:
response3

{'wall_time': 7.061087608337402,
 'input': 'What is the name of the cognac house that makes the main ingredient in The Hennchata?',
 'output': 'Hennessy.',
 'planner_log': "For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools. Available tools are: \n\nTools can be one of the following:\nWikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.\nLLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be 

In [10]:
print(response3["solver_log"])

Solve the following task or problem. To assist you, we provide some plans and corresponding evidences that might be helpful. Notice that some of these information contain noise so you should trust them with caution.

Are both The New Pornographers and Kings of Leon American rock bands?
Plan: Find out if The New Pornographers is an American rock band.
Evidence:
The New Pornographers are a Canadian indie rock band, formed in 1997 in Vancouver.
Plan: Find out if Kings of Leon is an American rock band.
Evidence:
Kings of Leon is an American rock band formed in Nashville, Tennessee, in 1999. The band is composed of brothers Caleb, Nathan and Jared Followill, and their cousin Matthew Followill.

Now begin to solve the task or problem. Repond directly with no extra words.
Are both The New Pornographers and Kings of Leon American rock bands?
Yes.


In [ ]:
from nodes.Worker import WikipediaWorker, DocStoreLookUpWorker
from utils.CustomDocstoreExplorer import CustomDocstoreExplorer
from langchain import Wikipedia
docstore = CustomDocstoreExplorer(Wikipedia())

In [17]:
import json
with open("prompts/prompts_naive.json") as f:
    data = json.load(f)

In [22]:
a

"Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nPlan: Search for more information about Colorado orogeny.\n#E1 = Wikipedia[Colorado orogeny]\nPlan: Look up for key information about eastern sector in search result.\n#E2 = LookUp[eastern sector]\nPlan: Find out the area that eastern sector of the Colorado orogeny extends into.\n#E4 = LLM[What is the name of the area that eastern sector of Colorado extends into? Given context: #E2]\nPlan: Search for more information about the area.\n#E5 = Wikipedia[#E4]\nPlan: Look up for the key information about elevation in search result.\n#E6 = LookUp[elevation]\nPlan: Find out the elevation range for the area.\n#E7 = LLM[What is elevation range for the area [#E4]? Given context: #E5]\n\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nPlan: Search for more information about Arthur's Magazine.\n#E1 = Wikipedia[Arthur's Magazine] \nPlan: Look up for key

In [2]:
from prompts import fewshots
fewshots.HOTPOTQA_PWS_BASE

"Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nPlan: Search for more information about Colorado orogeny.\n#E1 = Wikipedia[Colorado orogeny]\nPlan: Look up for key information about eastern sector in search result.\n#E2 = LookUp[eastern sector]\nPlan: Find out the area that eastern sector of the Colorado orogeny extends into.\n#E4 = LLM[What is the name of the area that eastern sector of Colorado extends into? Given context: #E2]\nPlan: Search for more information about the area.\n#E5 = Wikipedia[#E4]\nPlan: Look up for the key information about elevation in search result.\n#E6 = LookUp[elevation]\nPlan: Find out the elevation range for the area.\n#E7 = LLM[What is elevation range for the area [#E4]? Given context: #E5]\n\nQuestion: Which magazine was started first Arthur's Magazine or First for Women?\nPlan: Search for more information about Arthur's Magazine.\n#E1 = Wikipedia[Arthur's Magazine] \nPlan: Look up for key

In [5]:
from nodes.Worker import LLMWorker

tool = LLMWorker()
tool.run("Which magazine was started first Arthur's Magazine or First for Women? ")

"Arthur's Magazine."

In [6]:
print("For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools. Available tools are: \n\n")

For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools. Available tools are: 




## CoT

In [3]:
cot = CoT(fewshot=fewshots.HOTPOTQA_COT)
cot.run("Which magazine was started first Arthur's Magazine or First for Women?")

{'wall_time': 4.111898183822632,
 'input': "Answer following questions. Let's think step by step. Give your reasoning process, and then answer the question in a new line directly with no extra words.\nWhat is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nStep 1: Identify the eastern sector of the Colorado Orogeny\nThe Colorado Orogeny refers to a series of mountain-building events that occurred in the Rocky Mountains, primarily in the present-day U.S. state of Colorado. The eastern sector of the Colorado Orogeny likely refers to the easternmost part of this mountain range, which includes the Front Range, Wet Mountains, and the Sangre de Cristo Mountains.\nStep 2: Determine the elevation range of the eastern sector\nThe Front Range, Wet Mountains, and Sangre de Cristo Mountains have varying elevations. The Front Range has elevations ranging from around 5,000 feet to over 14,000 feet (1,500 meters to 4,300 meters). The Wet Mountains have 

## IO

In [7]:
io = IO()
io.run('In what city and state was the vocalist for  El Grupo Nuevo de Omar Rodriguez Lopez born?')

{'wall_time': 0.8477969169616699,
 'input': 'Answer following questions. Respond directly with no extra words.\n\nIn what city and state was the vocalist for  El Grupo Nuevo de Omar Rodriguez Lopez born?\n',
 'output': '\nSan Juan, Puerto Rico',
 'prompt_tokens': 37,
 'completion_tokens': 6,
 'total_tokens': 43,
 'total_cost': 0.0008600000000000001}

In [11]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("question-answering-state-of-the-union")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/billxbf/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--question-answering-state-of-the-union-a7e5a3b2db4f440d/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
len(dataset)

11

In [8]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")
llm = OpenAI(temperature=0)
chain = SQLDatabaseChain(llm=llm, database=db, input_key="question")

## State of the Union

In [16]:
from langchain.document_loaders import TextLoader
loader = TextLoader("data/docs/state_of_the_union.txt")

In [18]:
from langchain.indexes import VectorstoreIndexCreator
vectorstore = VectorstoreIndexCreator().from_loaders([loader]).vectorstore

Using embedded DuckDB without persistence: data will be transient


In [19]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [20]:
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever(), input_key="question")

In [21]:
chain

RetrievalQA(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fc265b082e0>, verbose=False, combine_documents_chain=StuffDocumentsChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fc265b082e0>, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fc265b082e0>, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x

In [25]:
predictions = chain.apply(dataset)

In [24]:
from langchain.evaluation.qa import QAEvalChain

In [26]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="result")

In [27]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [36]:
predictions

[{'question': 'What is the purpose of the NATO Alliance?',
  'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
  'result': ' The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
  'grade': ' CORRECT'},
 {'question': 'What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?',
  'answer': 'The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.',
  'result': " The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs. They are also joining with European allies to find and seize the oligarchs' yachts, luxury apartments, and private jets, and going after criminals who stole billions in relief money meant for small businesses and millions of Americans.",
  'grade': ' INCORRECT'},
 {'question': 'What is the American Rescue Plan and how did it help Amer

In [44]:
llm_accuracy_score("What is the name of the cognac house that makes the main ingredient in The Hennchata?", "Hennessy", "FUCK")

0

In [30]:
eval_chain

QAEvalChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fc265b082e0>, verbose=False, prompt=PromptTemplate(input_variables=['query', 'result', 'answer'], output_parser=None, partial_variables={}, template="You are a teacher grading a quiz.\nYou are given a question, the student's answer, and the true answer, and are asked to score it as either CORRECT or INCORRECT.\n\nExample Format:\nQUESTION: question here\nSTUDENT ANSWER: student's answer here\nTRUE ANSWER: true answer here\nGRADE: CORRECT or INCORRECT here\n\nPlease remember to grade them based on being factually accurate. Begin!\n\nQUESTION: {query}\nSTUDENT ANSWER: {result}\nTRUE ANSWER: {answer}\nGRADE:", template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fc265b082e0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', tem

In [32]:
predictions

[{'question': 'What is the purpose of the NATO Alliance?',
  'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
  'result': ' The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
  'grade': ' CORRECT'},
 {'question': 'What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?',
  'answer': 'The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.',
  'result': " The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs. They are also joining with European allies to find and seize the oligarchs' yachts, luxury apartments, and private jets, and going after criminals who stole billions in relief money meant for small businesses and millions of Americans.",
  'grade': ' INCORRECT'},
 {'question': 'What is the American Rescue Plan and how did it help Amer

In [10]:
agent = PWS_Base(fewshot=fewshots.HOTPOTQA_PWS_BASE)
agent.run("Pibb Xtra and Pet, Inc., are made in which country?")

{'wall_time': 5.717400312423706,
 'input': 'Pibb Xtra and Pet, Inc., are made in which country?',
 'output': 'USA.',
 'planner_log': "For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools. Available tools are: \n\nTools can be one of the following:\nWikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.\nLLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.\n\nBegin! Describe you

In [12]:
from nodes.Worker import WikipediaWorker
worker = WikipediaWorker()
worker.run("Leo Decaprio")

"Leonardo Wilhelm DiCaprio (, ; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors.\nBorn in Los Angeles, DiCaprio began his career in the late 1980s by appearing in television commercials. In the early 1990s, he had recurring roles in various television shows, such as the sitcom Parenthood, and had his first major film part as author Tobias Wolff in This Boy's Life (1993). He received critical acclaim and his first Academy Award and Golden Globe Award nominations for his performance as a developmentally disabled boy in What's Eating Gilbert Grape (1993). DiCaprio achieved international stardom with the star-crossed rom

In [4]:
agent.run("Which magazine was started first Arthur's Magazine or First for Women?")

{'wall_time': 10.688284158706665,
 'input': "Which magazine was started first Arthur's Magazine or First for Women?",
 'output': "Arthur's Home Magazine began in 1861, so it was started first.",
 'planner_log': "For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools.\n\nTools can be one of the following:\nLLM[input]: A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.\nGoogle[input]: Worker that searches results from Google. Useful when you need to find short and general answers about a specific topic. Input should be a search query.\nCalculator[input]: A calculator that can compute arithmetic expressions. Useful when you need t

In [7]:
agent = PWS_Extra(fewshot=fewshots.HOTPOTQA_PWS_EXTRA)
response = agent.run("Ikivo Animator integrates with a software application developed by which company?")

In [8]:
response

{'wall_time': 8.608613729476929,
 'input': 'Ikivo Animator integrates with a software application developed by which company?',
 'output': 'Hypermedia.',
 'planner_log': "For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools.\n\nTools can be one of the following:\nWikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.\nGoogle[input]: Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. Input should be a search query.\nCalculator[input]: A calculator that can compute arithme

In [10]:
print(response["planner_log"])

For the following tasks, make plans that can solve the problem step-by-step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E (#E1, #E2, ...) that can be called by later tools.

Tools can be one of the following:
Wikipedia[input]: Worker that search for similar page contents from Wikipedia. Useful when you need to get holistic knowledge about people, places, companies, historical events, or other subjects. The response are long and might contain some irrelevant information. Input should be a search query.
Google[input]: Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. Input should be a search query.
Calculator[input]: A calculator that can compute arithmetic expressions. Useful when you need to perform math calculations. Input should be a mathematical expression
LLM[input]: A pretrained LLM like yourself. Useful when you need t

In [11]:
print(response["solver_log"])

Solve the following task or problem. To assist you, we provide some plans and corresponding evidences that might be helpful. Notice that some of these information contain noise so you should trust them with caution.

Ikivo Animator integrates with a software application developed by which company?
Plan: Search for more information about Ikivo Animator.
Evidence:
Could not find [Ikivo Animator]. Similar: ['Hypermedia']
Plan: Find out the company that developed Ikivo Animator.
Evidence:
Hypermedia
Plan: Check on Google to see if the company is correct.
Evidence:
Software applications, such as Ikivo Animator and Inkscape, simplify the development of hypermedia content based on SVG. Embedded devices, such as the ...

Now begin to solve the task or problem. Respond directly with no extra words.
Ikivo Animator integrates with a software application developed by which company?
Hypermedia.
